In [ ]:
# Dependencias

!pip install -q torch==2.3.0+cu118 torchvision==0.18.0+cu118 torchaudio==2.3.0 --index-url https://download.pytorch.org/whl/cu118
!pip install torch torchvision 
!pip install segment-anything timm tqdm matplotlib


In [ ]:
# Importar librerías básicas
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import timm  # EfficientNet disponible en `timm`
import os

In [ ]:
# CONFIGURACIÓN DEL DISPOSITIVO
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

In [ ]:
DATASET_PATH = "/kaggle/input/frutas-nuevas/Frutas-seleccionadas"
train_path = os.path.join(DATASET_PATH, "Training")
val_path = os.path.join(DATASET_PATH, "Test")

In [ ]:
# --- Definir transformaciones ---
from torchvision import transforms

from torchvision.models import EfficientNet_B3_Weights
# definir transformaciones
weights = EfficientNet_B3_Weights.DEFAULT
imagenet_transforms = weights.transforms()  # ✅ define it outside the Compose

# Normalización estándar de ImageNet (usada por EfficientNet)
#imagenet_mean = [0.485, 0.456, 0.406]
#imagenet_std = [0.229, 0.224, 0.225]

# Data augmentation para entrenamiento
transform_train = transforms.Compose([
    transforms.Resize((300, 300)),
    transforms.RandomResizedCrop(300, scale=(0.8, 1.0)),
    transforms.RandomRotation(degrees=(0, 360)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomAffine(degrees=15, translate=(0.1, 0.1), scale=(0.9, 1.1), shear=10),
    transforms.ColorJitter(brightness=0.3, contrast=0.3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                         std=[0.229, 0.224, 0.225])
])

# Validación (sin data augmentation)
transform_val = transforms.Compose([
    transforms.Resize((300, 300)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                         std=[0.229, 0.224, 0.225])
])


In [ ]:
# ---  Cargar conjunto de datos ---

train_dataset = datasets.ImageFolder(os.path.join(DATASET_PATH, "Training"), transform=transform_train)
val_dataset = datasets.ImageFolder(os.path.join(DATASET_PATH, "Test"), transform=transform_val)

# ---  Crear DataLoaders ---
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_loader  = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)

# Mostrar 5 imagenes random del dataset y tamaño de carpetas Training y Test
print(f"Training size: {len(train_dataset)} | Test size: {len(val_dataset)}")

In [ ]:
# MODELO + FINE-TUNING

from torchvision.models import efficientnet_b3, EfficientNet_B3_Weights
num_classes = len(train_dataset.classes)

model = efficientnet_b3(weights=EfficientNet_B3_Weights.IMAGENET1K_V1)

# Congelar primeras capas
for param in model.parameters():
    param.requires_grad = False

# Modificar la última capa
model.classifier[1] = nn.Sequential(
    nn.Linear(model.classifier[1].in_features, 512),
    nn.ReLU(),
    nn.Dropout(0.3),  # Regularización
    nn.Linear(512, num_classes)
)

model = model.to(device)

In [ ]:
# OPTIMIZACIÓN Y FUNCIÓN DE PÉRDIDA

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.classifier.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)

In [ ]:
# ENTRENAMIENTO CON GUARDADO DEL MEJOR MODELO

epochs = 5 
best_val_acc = 0.0  # Inicializar mejor precisión en validación
# Listas para guardar métricas
train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []

history = {"train_losses": [], "val_losses": [], "train_accuracies": [], "val_accuracies": []}
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    running_corrects = 0
    total = 0

    pbar = tqdm(train_loader, desc=f"Entrenando Epoch {epoch+1}/{epochs}", leave=False)
    for inputs, labels in pbar:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        running_corrects += (preds == labels).sum().item()
        total += labels.size(0)

        pbar.set_postfix(loss=loss.item())

    epoch_loss = running_loss / total
    epoch_acc = 100 * (running_corrects / total)
    train_losses.append(epoch_loss)
    train_accuracies.append(epoch_acc)

    # Validación
    model.eval()
    val_running_loss = 0.0
    val_running_corrects = 0
    val_total = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            val_running_corrects += (preds == labels).sum().item()
            val_total += labels.size(0)

    
    val_epoch_loss = val_running_loss / val_total
    val_epoch_acc = 100 * (val_running_corrects / val_total)
    val_losses.append(val_epoch_loss)
    val_accuracies.append(val_epoch_acc)
    scheduler.step()

    
    print(f"Epoch {epoch+1}/{epochs} | "
          f"Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.2f}% | "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.2f}%")
    # # Guardar el modelo si la precisión en validación mejora
    
    if val_epoch_acc > best_val_acc:
        best_val_acc = val_epoch_acc
        
        save_dir = "/kaggle/working/models"

        # Crear la carpeta si no existe
        os.makedirs(save_dir, exist_ok=True)

        # Ruta para guardar el modelo
        best_model_path = os.path.join(save_dir, "best_efficientnet_b3.pth")
        torch.save(model.state_dict(), best_model_path)
        print(f"✅ Nuevo mejor modelo guardado con {best_val_acc:.2f}% de precisión en validación.")
        print(f"✅ Modelo guardado en {best_model_path}")

history["train_losses"].append(epoch_loss)
history["train_accuracies"].append(epoch_acc)
history["val_losses"].append(val_epoch_loss)
history["val_accuracies"].append(val_epoch_acc)


In [ ]:
# ---  Graficar resultados ---
plt.figure(figsize=(12,5))

plt.subplot(1,2,1)
plt.plot(train_losses, label='Pérdida Entrenamiento')
plt.plot(val_losses, label='Pérdida Validación')
plt.xlabel('Época')
plt.ylabel('Pérdida')
plt.title('Pérdida durante Entrenamiento y Validación')
plt.legend()

plt.subplot(1,2,2)
plt.plot(train_accuracies, label='Precisión Entrenamiento')
plt.plot(val_accuracies, label='Precisión Validación')
plt.xlabel('Época')
plt.ylabel('Precisión (%)')
plt.title('Precisión durante Entrenamiento y Validación')
plt.legend()

plt.show()

In [ ]:
##  ---------------    Matriz de Confusion ---------------
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import numpy as np

# --- Obtener predicciones en test ---
all_preds = []
all_labels = []

model.eval()
with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# --- Clasificación por clase ---
print(classification_report(all_labels, all_preds, target_names=val_dataset.classes))

# --- Matriz de confusión ---
cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(12,10))
sns.heatmap(cm, annot=True, fmt='d', xticklabels=val_dataset.classes, yticklabels=val_dataset.classes, cmap='Blues')
plt.xlabel("Predicho")
plt.ylabel("Real")
plt.title("Matriz de Confusión en Validación")
plt.show()

In [ ]:
from torchvision import datasets

train_dataset = datasets.ImageFolder(root="/kaggle/input/frutas-nuevas/Frutas-seleccionadas/Training")
print("📝 Orden de las clases:", train_dataset.classes)